In [1]:
# Update sbml_annotation so that specificity is weighted based on qualifier
import itertools
import libsbml
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import sys
sys.path.append(os.path.join(os.getcwd(),"" '..'))
from SBMate import constants as cn
from SBMate import sbmate
from SBMate import sbml_annotation2 as sa
BIOMD_12 = 'BIOMD0000000012.xml'
BIOMD_13 = 'BIOMD0000000013.xml'
BIOMD_15 = 'BIOMD0000000015.xml'

In [2]:
raw_annotation = sa.RawSBMLAnnotation(BIOMD_12)

In [3]:
a = dict()
if a:
  print("a is dict")
print(type(a))

<class 'dict'>


In [4]:
for one_anot in raw_annotation.str_annotation[:3]:
    print(one_anot)
    print()

ObjectAnnotation(id='BIOMD0000000012', object_type=<class 'libsbml.Model'>, annotation={'isVersionOf': ['<bqbiol:isVersionOf>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/obo.go/GO:0040029"/>\n        </rdf:Bag>\n      </bqbiol:isVersionOf>']})

ObjectAnnotation(id='cell', object_type=<class 'libsbml.Compartment'>, annotation={'is': ['<bqbiol:is>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/obo.go/GO:0005623"/>\n        </rdf:Bag>\n      </bqbiol:is>']})

ObjectAnnotation(id='PX', object_type=<class 'libsbml.Species'>, annotation={'is': ['<bqbiol:is>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/uniprot/P03023"/>\n        </rdf:Bag>\n      </bqbiol:is>']})



In [5]:
processed_annotation = sa.SBMLAnnotation(BIOMD_12)

In [6]:
processed_annotation.annotations['PX']

{'go': None,
 'sbo': ['SBO:0000252'],
 'chebi': None,
 'kegg_species': None,
 'kegg_process': None,
 'uniprot': ['P03023'],
 'object_id': 'PX',
 'object_type': libsbml.Species}

### Test script (test_sbml_annotation2.py)

In [7]:
reader = libsbml.SBMLReader()
document = reader.readSBML(os.path.join(cn.TEST_DIR, BIOMD_12))
sbml_model = document.getModel()
raw_sbml_annotation = sa.RawSBMLAnnotation(input_file=os.path.join(cn.TEST_DIR, BIOMD_12))
# BIOMD13 is for checking incompelte annotation 
incomplete_document = reader.readSBML(os.path.join(cn.TEST_DIR, BIOMD_15))
incompelte_sbml_model = incomplete_document.getModel()
incomplete_raw_sbml_annotation = sa.RawSBMLAnnotation(input_file=os.path.join(cn.TEST_DIR, BIOMD_15))

In [8]:
# testGetSBOAnnotation
# reaction1_sbo_annotation = raw_sbml_annotation.getSBOAnnotation(sbml_model.getReaction('Reaction1'))
# print(reaction1_sbo_annotation.id)
# print(reaction1_sbo_annotation.object_type)
# print(reaction1_sbo_annotation.annotation)

# testGetOntAnnotation
reaction1_annotation = raw_sbml_annotation.getOntAnnotation(sbml_model.getReaction('Reaction1'))
reaction1_str_annotation = '<bqbiol:isVersionOf>\n        <rdf:Bag>\n          <rdf:li rdf:resource="http://identifiers.org/obo.go/GO:0006402"/>\n        </rdf:Bag>\n      </bqbiol:isVersionOf>'
print(reaction1_annotation[0]=='Reaction1')
print(reaction1_annotation[1]==libsbml.Reaction)
print(reaction1_annotation[2]['isVersionOf'][0]==reaction1_str_annotation)

True
True
True


In [9]:
px_annotation = raw_sbml_annotation.getOntAnnotation(sbml_model.getSpecies('PX'))
px_str_annotation = '<bqbiol:is>\n  <rdf:Bag>\n    <rdf:li rdf:resource="http://identifiers.org/uniprot/P03023"/>\n  </rdf:Bag>\n</bqbiol:is>'
print(px_annotation.id=='PX')
print(px_annotation.object_type==libsbml.Species)
print(px_annotation.annotation['is'][0]==px_str_annotation)

True
True
False


In [10]:
no_ont_annotation = incomplete_raw_sbml_annotation.getOntAnnotation(incompelte_sbml_model.getSpecies('ATP'))
print(no_ont_annotation.id=='ATP')
print(no_ont_annotation.object_type==libsbml.Species)
print(no_ont_annotation.annotation=={}) 

True
True
True


In [11]:
sbml_annotation = sa.SBMLAnnotation(file=os.path.join(cn.TEST_DIR, BIOMD_12))
input_annotation = '<bqbiol:isVersionOf>\n  <rdf:Bag>\n    <rdf:li rdf:resource="http://identifiers.org/obo.go/GO:0006402"/>\n  </rdf:Bag>\n</bqbiol:isVersionOf>'
    

In [13]:
model_annotation = sbml_annotation.getAnnotationDictByQualifier('BIOMD0000000012')

In [14]:
model_annotation

{'isVersionOf': [('obo.go', 'GO:0040029')],
 'object_id': 'BIOMD0000000012',
 'object_type': libsbml.Model}

In [17]:
px_annotation = sbml_annotation.getAnnotationDictByQualifier('PX')
px_annotation

{'is': [('uniprot', 'P03023'), ('sbo', 'SBO:0000252')],
 'object_id': 'PX',
 'object_type': libsbml.Species}